In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    "User-Agent": "Mozilla/5.0"
}

def crawl_coupang(url):
    print("크롤링:", url)
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")

    items = soup.select("li.carousel-item.fw-group")
    results = []

    for item in items:
        # 이미지
        img_tag = item.select_one("div.item-image img")
        img = img_tag.get("src") if img_tag else None
        alt = img_tag.get("alt") if img_tag else None

        # 제품명
        name_tag = item.select_one("div.item-info-wrap .name")
        name = name_tag.get_text(strip=True) if name_tag else alt

        # 가격
        price_tag = item.select_one(".price-value")
        price = price_tag.get_text(strip=True) if price_tag else None

        results.append({
            "product_name": name,
            "price": price,
            "image": img,
            "alt_name": alt
        })

    return pd.DataFrame(results)


url = "https://www.coupang.com/"
df = crawl_coupang(url)

print(df.head())
print("총 개수:", len(df))

크롤링: https://www.coupang.com/


KeyboardInterrupt: 

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def get_driver():
    options = Options()
    # options.add_argument("--headless=new")   # 필요하면 주석 해제
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)


def crawl_coupang_list(url, scroll_count=5, scroll_pause=2.0):
    driver = get_driver()
    driver.get(url)

    # --- 스크롤 다운 ---
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # --- HTML 가져오기 ---
    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    # 🔥 네 DOM 기반 정확한 selector
    items = soup.select("li.carousel-item")

    rows = []
    for it in items:
        # 1) 상품 이미지 / 이름
        img = it.select_one("div.item-image img")
        name = img.get("alt").strip() if img and img.get("alt") else None

        # 2) 상세 링크
        a_tag = it.select_one("a")
        href = a_tag.get("href") if a_tag and a_tag.get("href") else None
        if href and href.startswith("/"):
            href = "https://www.coupang.com" + href

        # 3) 가격 (쿠팡 기본 selector)
        price_el = it.select_one("strong.price-value")
        price = price_el.get_text(strip=True) if price_el else None

        # 4) 브랜드
        brand_el = it.select_one("span.seller, span.brand")
        brand = brand_el.get_text(strip=True) if brand_el else None

        # 5) 평점
        rating_el = it.select_one("em.rating")
        rating = rating_el.get_text(strip=True) if rating_el else None

        # 6) 리뷰 수
        review_el = it.select_one("span.rating-total-count")
        review_cnt = (
            review_el.get_text(strip=True).strip("()")
            if review_el
            else None
        )

        if not name:
            continue  # 이름 없는 카드면 skip

        rows.append({
            "name": name,
            "brand": brand,
            "price": price,
            "rating": rating,
            "review_count": review_cnt,
            "link": href
        })

    return pd.DataFrame(rows)

In [3]:
url = "https://www.coupang.com/"
df = crawl_coupang_list(url, scroll_count=8, scroll_pause=2.0)

print(df.head())
print("총 개수:", len(df))

df.to_csv("coupang_list.csv", index=False)

Empty DataFrame
Columns: []
Index: []
총 개수: 0


In [4]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_driver():
    options = Options()
    # options.add_argument("--headless=new")  # 필요하면 켜기
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)

def crawl_coupang_search(url, scroll_count=5, scroll_pause=2.0):
    driver = get_driver()
    driver.get(url)

    # ----- 아래로 스크롤해서 상품 더 로딩 -----
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    # 🔥 검색/카테고리 상품 카드 선택자
    items = soup.select("li.search-product")

    rows = []
    for it in items:
        # 상품명
        name_el = it.select_one("div.name")
        name = name_el.get_text(strip=True) if name_el else None

        # 상세 링크
        a_tag = it.select_one("a")
        href = a_tag.get("href") if a_tag and a_tag.get("href") else None
        if href and href.startswith("/"):
            href = "https://www.coupang.com" + href

        # 가격
        price_el = it.select_one("strong.price-value")
        price = price_el.get_text(strip=True) if price_el else None

        # 브랜드
        brand_el = it.select_one("div.prod-brand-name")
        brand = brand_el.get_text(strip=True) if brand_el else None

        # 평점
        rating_el = it.select_one("em.rating")
        rating = rating_el.get_text(strip=True) if rating_el else None

        # 리뷰 수
        review_el = it.select_one("span.rating-total-count")
        review_cnt = (
            review_el.get_text(strip=True).strip("()")
            if review_el else None
        )

        if not name:
            continue

        rows.append({
            "name": name,
            "brand": brand,
            "price": price,
            "rating": rating,
            "review_count": review_cnt,
            "link": href,
        })

    return pd.DataFrame(rows)

In [5]:
url = "https://www.coupang.com/np/search?q=%EC%96%B4%EC%84%B1%EC%B4%88+%ED%86%A0%EB%84%88+%ED%8C%A8%EB%93%9C&listSize=60"

df = crawl_coupang_search(url, scroll_count=5, scroll_pause=2.0)

print(df.head())
print("총 개수:", len(df))

df.to_csv("coupang_list.csv", index=False)

Empty DataFrame
Columns: []
Index: []
총 개수: 0


In [6]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_driver():
    options = Options()
    # 필요하면 주석 해제해서 창 안 뜨게
    # options.add_argument("--headless=new")
    options.add_argument("--start-maximized")
    return webdriver.Chrome(options=options)

def crawl_coupang_search(url, scroll_count=5, scroll_pause=2.0):
    driver = get_driver()
    driver.get(url)

    # ----- 아래로 여러 번 스크롤해서 상품 더 로딩 -----
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    # 🔥 검색/카테고리 페이지용 상품 카드 선택자
    items = soup.select("li.search-product")

    rows = []
    for it in items:
        name_el = it.select_one("div.name")
        name = name_el.get_text(strip=True) if name_el else None
        if not name:
            continue

        a_tag = it.select_one("a")
        href = a_tag.get("href") if a_tag and a_tag.get("href") else None
        if href and href.startswith("/"):
            href = "https://www.coupang.com" + href

        price_el = it.select_one("strong.price-value")
        price = price_el.get_text(strip=True) if price_el else None

        brand_el = it.select_one("div.prod-brand-name")
        brand = brand_el.get_text(strip=True) if brand_el else None

        rating_el = it.select_one("em.rating")
        rating = rating_el.get_text(strip=True) if rating_el else None

        review_el = it.select_one("span.rating-total-count")
        review_cnt = review_el.get_text(strip=True).strip("()") if review_el else None

        rows.append({
            "name": name,
            "brand": brand,
            "price": price,
            "rating": rating,
            "review_count": review_cnt,
            "link": href,
        })

    return pd.DataFrame(rows)

In [7]:
url = "https://www.coupang.com/np/search?q=%EC%96%B4%EC%84%B1%EC%B4%88+%ED%86%A0%EB%84%88+%ED%8C%A8%EB%93%9C&listSize=60"

df = crawl_coupang_search(url, scroll_count=5, scroll_pause=2.0)

print(df.head())
print("총 개수:", len(df))

df.to_csv("coupang_list.csv", index=False)

Empty DataFrame
Columns: []
Index: []
총 개수: 0
